In [11]:
import numpy as np
import matplotlib.pyplot as plt

import package_DBR
from package_DBR import myRound, SelectPath_RT, Delay_RT, FO_RT


import Package18155_18316
from importlib import reload
Package18155_18316 = reload(Package18155_18316)

from Package18155_18316 import LeadLag_RT, PID_RT, IMC_Tuning


In [12]:
help(IMC_Tuning)

Help on function IMC_Tuning in module Package18155_18316:

IMC_Tuning(K, Tlag1, Tlag2=0.0, theta=0.0, gamma=0.5, process='FOPDT_PID')
    This function computes the IMC PIC tuning parameters for FOPDT and SOPDT processes.
    :K: process gain [/]
    :Tlag1: First lag time constant [s]
    :Tlag2: second lag timae constant[s] can be optional _default=0.0
    :theta: delay[s] _default=0.0
    :gamma:used to computed the desired closed-loop time constant TCLP[s] _default=0.5
    :process: _default=FOPDT_PID
            FOPDT_PID: First Order Plus Dead Time for PID control (case H)
            SOPDT_PID: Second Order Plus Dead Time for PID control (case I)
    
    
    
    
    :return: Parameters of the PID controller : Kc, Ti, Td



In [13]:
help(LeadLag_RT)

Help on function LeadLag_RT in module Package18155_18316:

LeadLag_RT(MV, Kp, Tlead, Tlag, Ts, PV, PVinit=0, method='EBD')
    The function "LeadLag_RT" needs to be included in a "for or while loop".
    
    :MV: input vector
    :Kp: process gain
    :Tlead: lead time constant [s]
    :Tlag: lag time constant [s]
    :Ts: sampling period [s]
    :PV: output vector
    :PVInit: (optional: default value is 0)
    :method: discretisation method (optional: default value is 'EBD')
        EBD: Euler Backward difference
        EFD: Euler Forward difference
        TRAP: Trapezoïdal method
    
    The function "LeadLag_RT" appends a value to the output vector "PV".
    The appended value is obtained from a recurrent equation that depends on the discretisation method.



In [14]:
TSim = 100
Ts = 0.1
N = int(TSim/Ts) + 1
#Working Point
MV0=50
DV0=50
PV0=74.72
#p= influence of the manipulated variable MV
#d= influence of the distrubance variable DV
#parameters of input-output
Kp=0.5757476550252816
T1p=99.23037687849623
T2p=0.00016543208442899084
thetap=7.107567046646536

#parameters of the distrubance
Kd=0.3693921329180759
T1d=45.12575781307914
T2d=46.415063698602864
thetad=6.742745758036814

#Parameters of the PID controller 
gamma=0.7
Kc, Ti, Td= IMC_Tuning(Kp, T1p, T2p, thetap, gamma, 'SOPDT_PID')
alpha= 1 
MVMin=0
MVMax=100


# Path for signals
ManPath={0:True, 500:False, TSim:False} #auto mode
MVManPath={0:MV0, TSim:MV0}
SPPath = {0: PV0, 500: PV0-10, TSim:PV0-10}
DVPath = {0: DV0, 500: DV0+10, TSim:DV0+10}
FFActif=True
ManFF=False

In [15]:
t=[]
SP=[]
MV=[]
Man=[]
MVMan=[]
MVFF=[]
MVP=[]
MVI=[]
MVD=[]
DV=[]
PV=[]
E=[]

MVDelayp=[]
MVDelayd=[]
MVFFLL1=[]

PV1p=[]
PV2p=[]

PV1d=[]
PV2d=[]

MVFFDelay=[]
MVdelayPro=[]
MVdelayDist=[]


for i in range(0,N):
    t.append(i*Ts)
    #Signals SP - DV
    SelectPath_RT(SPPath,t,SP)
    SelectPath_RT(DVPath,t,DV)

    #FeedForward
    Delay_RT(DV-DV0*np.ones_like(DV),np.max([thetad-thetap,0]),Ts,MVFFDelay)
    LeadLag_RT(MVFFDelay, -Kd/Kp, T1p, T1d, Ts, MVFFLL1)
    if FFActif:
        LeadLag_RT(MVFFLL1,1,T2p,T2d,Ts,MVFF)
    else: 
        LeadLag_RT(MVFFLL1,1,T2p,T2d,Ts,MVFF)#check

    #PID mode
    SelectPath_RT(ManPath,t,Man)
    SelectPath_RT(MVManPath,t, MVMan)

    #PID action
    PID_RT(SP,PV,Man,MVMan,MVFF, Kc, Ti,Td,alpha, Ts,MVMin, MVMax,MV,MVP,MVI,MVD,E,ManFF,PV0)


    #Input-Output dynamics P(s)
    Delay_RT(MV,thetap,Ts,MVDelayp,MV0)
    FO_RT(MVDelayd,Kd,T1d,Ts,PV1d,0)
    FO_RT(PV1p,1,T2p,Ts,PV2p,0)

    #Disturbance dynamics D(s)
    Delay_RT(DV-DV0*np.ones_like(DV),thetad,Ts,MVDelayd,0)
    FO_RT(MVDelayp,Kp,T1p,Ts,PV1p,0)
    FO_RT(PV1d,1,T2d,Ts,PV2d,0) #with the output of the first FP_RT
    
    PV.append(PV2p[-1]+PV2d[-1]+PV0-Kp*MV0)  #PV0-Kp*MV0=offset

In [17]:
ManInit=[int(x) for x in Man]
#print(Man)
fig, (ax1, ax2, ax3, ax4)=plt.subplots(4,1)
fig.set_figheight(22)
fig.set_figheight(22)

l1,=ax1.step([0,t[-1]],[0,100],'k-', linewidth=2, label='Man', where='post')
ax1.set_ylabel('Value of Man [0 or 1]')
ax1.set_title('Closed Loop response with PID controller et feedforward')
ax1.legend(loc='best')


l2,=ax2.step([0,t[-1]],[0,100],'b-', linewidth=2, label='MV', where='post')
ax2.set_ylabel('Value of MV [%]')
ax2.legend(loc='best')

l3,=ax3.step([0,t[-1]],[0,100],'y-', linewidth=2, label='SP', where='post')
ax3.legend(loc='best')

l4,=ax3.step([0,t[-1]],[0,100],'g-', linewidth=2, label='PV', where='post')
ax3.set_ylabel('Value of PV [°C]')
ax3.legend(loc='best')

l5,=ax4.step([0,t[-1]],[0,100],'r-', linewidth=2, label='DV', where='post')
ax4.set_xlabel('Time [s]')
ax4.set_ylabel('Value of DV [%]')
ax4.legend(loc='best')


l1.set_data(t,ManInit)
l2.set_data(t,MV)
l3.set_data(t,SP)
l4.set_data(t,PV)
l5.set_data(t,DV)

ax1.set_xlim(0,t[-1]+1)
ax2.set_xlim(0,t[-1]+1)
ax3.set_xlim(0,t[-1]+1)
ax4.set_xlim(0,t[-1]+1)

ax1.set_ylim(-0.2,1.1)
#ax2.set_ylim(myRound(np.min(MV),5)-5, myRound(np.max(MV),5)+5)


(-0.1, 1.1)

ValueError: shape mismatch: objects cannot be broadcast to a single shape

<Figure size 432x1584 with 4 Axes>